In [2]:
from qchart import start_listener, listener_is_running
from qchart.client import DataSender
from datetime import datetime
import copy
import time
import numpy as np

def get_new_id():
    # use this to tag datasets
    t = datetime.now().strftime('%Y-%M-%d_%H-%M-%S-%f')
    return f'test_nik_{t}'

In [5]:
from functools import partial

def sine_model(x, freq: float, phase: float, noise: float=1):
    """
    Returns a generator sampling a sine wave. The sine wave has an 
    amplitude of 1 and some noise
    """

    rad_phase = np.pi*phase/180.0
    model = np.sin(2*np.pi*x + rad_phase)
    noise = np.random.rand(*x.shape)*noise
    return np.array(model + noise)

def gauss_model(x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    while True:
        (x, y) = yield
        model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
        noise = np.random.randn()*noise
        yield (model + noise)

In [7]:
if not listener_is_running():
    start_listener()

starting qchart listener at c:\users\nikha\repos\qchart\listener_start.py ...
qchart listener successfully started.


### 1D data in one shot

In [57]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'A'
    },
    'y_param': {
        'values': [],
        'unit': '',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)

x_vals = np.linspace(3,4,100)
y_vals = get_sin1(x_vals)

sndr = DataSender(get_new_id())

data = copy.deepcopy(data_structure)
data['x_param']['values'] = x_vals
data['y_param']['values'] = y_vals

sndr.send_data(data)

### 1D data in 1 shot with 2 outputs

In [58]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    },
    'y_param_2': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)
get_sin2 = partial(sine_model, freq=12, phase = 99, noise=0.02)

x_vals = np.linspace(3,4,100)

sndr = DataSender(get_new_id())

data = copy.deepcopy(data_structure)
data['x_param']['values'] = x_vals
data['y_param']['values'] = get_sin1(x_vals)
data['y_param_2']['values'] = get_sin2(x_vals)

sndr.send_data(data)

### Stream 1D data

In [8]:
# template for sending data through DataSender
data_structure = {
    'x_param': {
        'values': [],
        'unit': 'mA'
    },
    'y_param': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    },
    'y_param_2': {
        'values': [],
        'unit': 'arb',
        'axes': ['x_param']
    }
}

# getter function
get_sin1 = partial(sine_model, freq=10, phase = 9, noise=0.1)
get_sin2 = partial(sine_model, freq=12, phase = 99, noise=0.02)

x_vals = np.linspace(0,10,100).reshape(2,-1)

sndr = DataSender(get_new_id())

for x in x_vals:
    data = copy.deepcopy(data_structure)
    data['x_param']['values'] = x
    data['y_param']['values'] = get_sin1(x)
    data['y_param_2']['values'] = get_sin2(x)

    time.sleep(1.0) # artifical delay to pretent like we're collecting data

    sndr.send_data(data)

In [66]:
x_vals

array([ 0.        ,  0.1010101 ,  0.2020202 ,  0.3030303 ,  0.4040404 ,
        0.50505051,  0.60606061,  0.70707071,  0.80808081,  0.90909091,
        1.01010101,  1.11111111,  1.21212121,  1.31313131,  1.41414141,
        1.51515152,  1.61616162,  1.71717172,  1.81818182,  1.91919192,
        2.02020202,  2.12121212,  2.22222222,  2.32323232,  2.42424242,
        2.52525253,  2.62626263,  2.72727273,  2.82828283,  2.92929293,
        3.03030303,  3.13131313,  3.23232323,  3.33333333,  3.43434343,
        3.53535354,  3.63636364,  3.73737374,  3.83838384,  3.93939394,
        4.04040404,  4.14141414,  4.24242424,  4.34343434,  4.44444444,
        4.54545455,  4.64646465,  4.74747475,  4.84848485,  4.94949495,
        5.05050505,  5.15151515,  5.25252525,  5.35353535,  5.45454545,
        5.55555556,  5.65656566,  5.75757576,  5.85858586,  5.95959596,
        6.06060606,  6.16161616,  6.26262626,  6.36363636,  6.46464646,
        6.56565657,  6.66666667,  6.76767677,  6.86868687,  6.96